In [ ]:
!pip install -U ultralytics

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11l.pt")

# Train the model
result = model.train(
    data="/kaggle/input/mammals-dataset/123123/data.yaml", 
    epochs=350, 
    optimizer='AdamW', 
    degrees=8,
    shear=4,
    fliplr=0.6,
    mixup=0.4,
    copy_paste=0.4,
    translate=0.35,
    patience=150,
    scale=0.7,
    imgsz=740)

In [ ]:
model.save('model.pt')